In [40]:
import numpy as np
import re

In [41]:
def getAngularError(R_exp, R_est):
    return np.abs(np.arccos(min(max((np.trace(R_exp.T @ R_est) - 1.0)/2, -1.0), 1.0)))


getAngularError(np.eye(3), np.array([[1, 0, 0], [0, -1, 0], [0, 0, 1]])) / np.pi * 180

90.0

In [42]:
def filter_list(arr, thred):
    idx = np.where(np.array(arr) <= thred)
    return np.array(arr)[idx]

def filter_idx(data, rot_thred=45, trans_thred=0.5):
    idx = []
    for i in range(len(data["rot_err"])):
        if data["rot_err"][i] < rot_thred and data["trans_err"][i] < trans_thred and data["runtime"][i]>0:
            idx.append(i)
    if len(idx) == 0:
        idx = [0]
    return np.array(idx)

In [43]:
import os

model_names = ["airplane", "bed", "chair", "desk", "guitar", "mantel", "monitor", "piano", "sofa", "stairs"]

test_dir = "../modelnet40_partial_overlap_test/"
original_dir = "../ModelNet40_subset/"

noise_levels = [0.0, 0.02, 0.04, 0.06, 0.08, 0.1]
noise_levels_2 = [0.02, 0.04, 0.06, 0.08, 0.1]
exact_recovery_rot = 5.0
exact_recovery_trans = 0.03

########################################################
algorithms = ['fls']
metrics = ['rot_err', 'trans_err', 'runtime', 'exact_recovery', 'failure']
labels = [r'$\bf{FLS(Ours)}$']

data = dict({})
for noise_level in noise_levels:
    data = dict({})
    for algo in algorithms:
        data[algo] = dict({})
        for metric in metrics:
            data[algo][metric] = []
########################################################


for model in model_names:
    file_list = os.listdir(original_dir + model + "/test/")
    for model_file in file_list:
        object = model_file.split('.')[0]
        filename = original_dir + model + "/test/" + object + ".off"
        # subprocess.run(["find", filename])

        # make dir for the current object
        run_dir = test_dir + object + "/"

        gt_T = np.array([float(ele) for ele in re.split(' |\n', open(run_dir + "gt_T.txt").read())[0:16]]).reshape(4,4)
        gt_R = gt_T[0:3, 0:3]
        gt_t = gt_T[0:3, 3]

        for algo in algorithms:
            opt_T = np.loadtxt(run_dir + algo + "_T.txt")
            opt_R = opt_T[0:3, 0:3]
            opt_t = opt_T[0:3, 3]
            runtime = np.loadtxt(run_dir + algo + "_time.txt")

            rot_err = getAngularError(gt_R, opt_R) / np.pi * 180 
            if np.isnan(rot_err):
                print('nan!')
            trans_err = np.linalg.norm(gt_t - opt_t)

            exact_recovery = 0
            if rot_err < exact_recovery_rot and trans_err < exact_recovery_trans and runtime>0:
                exact_recovery = 1

            failure = 0
            if rot_err > 45 or trans_err > 0.5 or runtime<0:
                failure = 1

            data[algo]['rot_err'].append(rot_err)
            data[algo]['trans_err'].append(trans_err)
            data[algo]['runtime'].append(runtime)
            data[algo]['exact_recovery'].append(exact_recovery)
            data[algo]['failure'].append(failure)

In [44]:
for algo in algorithms:
    print("\n================== {} ==================".format(algo))
    for metric in metrics:
        if metric == 'rot_err' or metric == 'trans_err':
            metric_mean = np.mean(np.array(data[algo][metric])[filter_idx(data[algo])])
            metric_std = np.std(np.array(data[algo][metric])[filter_idx(data[algo])])
        elif metric == 'runtime':
            metric_mean = np.mean(np.array(data[algo][metric])[np.where(np.array(data[algo][metric])>0)])
            metric_std = np.std(np.array(data[algo][metric])[np.where(np.array(data[algo][metric])>0)])
        else:
            metric_mean = failure_rate = np.mean(data[algo][metric]) * 100
            metric_std = 0.0
        print("{}: {}+-{}".format(metric, metric_mean, metric_std))


================== fls ==================
rot_err: 2.288941235499334+-1.6655668250906568
trans_err: 0.013758293325726468+-0.007504815380387825
runtime: 0.05253222+-0.015789497498387973
exact_recovery: 84.0+-0.0
failure: 8.0+-0.0
